In [1]:
import numpy as np
import os
import torch

from glob import glob
from skimage import io
from tensorboardX import SummaryWriter
from torch.utils.data.dataloader import DataLoader
from torch import nn
from tqdm import tqdm
from time import time

from src.batch_generator import BatchGenerator, WraptorDataLoader
from src.command import Command
from src.models import LocalTransferNet, GlobalTransferer, RefinementNet
from src.train_utils import (draw_scalar_value, draw_images, rgb2gray, infer_batch, 
                             inference_all_test_videos, load_all_test_videos, calculate_psnr_result)


from color_utils import rgb2lab_torch, lab2rgb_torch

In [2]:
os.environ['CUDA_VISIBLE_DEVICES']='1, 6'

experiment_dir = "experiments/train_refinement_useasfirstany_5framesahead_pretrained_local_amsgrad_mobilenet"
logs_directory = os.path.join(experiment_dir, "logs")
save_test_output_videos_dir =  os.path.join(experiment_dir,  "results_test_videos")
checkpoints_folder =  os.path.join(experiment_dir, "checkpoints")

board_port = 6104
board_timeout=24*60*60
batch_size = 4
img_shape = (128, 128, 3)
val_num = 5
iteration_num = 10000
dataset_dir = "datasets/Davis480/480p"
learning_rate = 1e-3 / 3
verbose_every_it = 10
save_models_every_it = 100

test_every_it = 100
back_shift = (1, 1)
frames_ahead = 5
use_as_first_only_first = False
global_feature_extractor_net = "mobilenet"

In [3]:
writer = SummaryWriter(logs_directory) 
board = Command('tensorboard --logdir=run1:{} --port {}'.format(logs_directory, board_port))
board.run()

In [4]:
dir_paths = [experiment_dir, logs_directory, checkpoints_folder, save_test_output_videos_dir]
for path in dir_paths:
    try:
        os.mkdir(path)
    except:
        pass

In [ ]:
local_transfer_net = nn.DataParallel(LocalTransferNet().double().cuda())
global_transferer = GlobalTransferer(net_type=global_feature_extractor_net)
refinement_net = nn.DataParallel(RefinementNet().double().cuda())

In [ ]:
train_bg = BatchGenerator(sorted(glob(dataset_dir + "/*"))[:-val_num], use_as_first_only_first=use_as_first_only_first, 
                          max_frames_ahead=frames_ahead, 
                          resize_shape=img_shape, need_crop=True)
train_loader = WraptorDataLoader(train_bg, batch_size=batch_size, shuffle=True)

val_bg = BatchGenerator(sorted(glob(dataset_dir + "/*"))[-val_num:], use_as_first_only_first=use_as_first_only_first, 
                          max_frames_ahead=frames_ahead, 
                          resize_shape=img_shape, need_crop=True)
val_loader = WraptorDataLoader(val_bg, batch_size=batch_size, shuffle=True)
test_videos_folders = sorted(glob(dataset_dir + "/*"))[-val_num:]

100%|██████████| 5/5 [00:00<00:00, 26.16it/s]


In [ ]:
local_transfer_net.load_state_dict(torch.load("experiments/pretrain_local_several_frames_back_highlr_amsgrad/checkpoints/7800/local_net_state_dict"))

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [ ]:
pretrain_local = False
end2end = False

In [ ]:
loss_function = torch.nn.L1Loss() #torch.nn.MSELoss() or torch.nn.L1Loss()
if pretrain_local:
    optimizer = torch.optim.Adam(list(local_transfer_net.parameters()), 
                                 lr=learning_rate, amsgrad=True)
else:
    if end2end:
        optimizer = torch.optim.Adam(list(local_transfer_net.parameters()) + list(refinement_net.parameters()), 
                                     lr=learning_rate, amsgrad=True)
    else:
        optimizer = torch.optim.Adam(list(refinement_net.parameters()), 
                                     lr=learning_rate, amsgrad=True)

In [12]:
start_it = 10000

In [ ]:
#I1, Ik_1, Ik
train_losses = []
for it in tqdm(range(start_it, start_it + iteration_num)):
    train_batch = next(iter(train_loader))
    refinement_output_lab, result_rgb_train, local_output_train, global_output_train = infer_batch(train_batch, 
                                                                                                   refinement_net, local_transfer_net, global_transferer,
                                                                                                   use_only_local=pretrain_local, 
                                                                                                   verbose= (it == 0))

    Ik_lab = rgb2lab_torch(train_batch[2].cuda(), use_gpu=True)
    
    if pretrain_local:
        train_loss = loss_function(refinement_output_lab, Ik_lab)
    else:
        train_loss = loss_function(refinement_output_lab[..., 1:], Ik_lab[..., 1:])
    optimizer.zero_grad()
    train_loss.backward()
    optimizer.step()
    
    if (it % verbose_every_it) == 0:
        val_batch = next(iter(val_loader))
        Ik_lab = rgb2lab_torch(val_batch[2].cuda(), use_gpu=True)
        refinement_output_lab, result_rgb_val, local_output_val, global_output_val = infer_batch(val_batch,
                                                                                                 refinement_net, local_transfer_net, global_transferer,
                                                                                                 use_only_local=pretrain_local,
                                                                                                 verbose = False)
        if pretrain_local:
            val_loss = loss_function(refinement_output_lab, Ik_lab)
        else:
            val_loss = loss_function(refinement_output_lab[..., 1:], Ik_lab[..., 1:])
        
        draw_scalar_value(writer, "losses", "train loss", train_loss.detach().cpu().numpy(), it)
        draw_scalar_value(writer, "losses", "val loss", val_loss.detach().cpu().numpy(), it)
        
        local_output_train = torch.clamp(local_output_train,  0, 1)
        local_output_val = torch.clamp(local_output_val,  0, 1)
        result_rgb_train = torch.clamp(result_rgb_train,  0, 1)
        result_rgb_val = torch.clamp(result_rgb_val,  0, 1)

        concat_res_train = torch.cat((global_output_train.cpu(), local_output_train.cpu(), result_rgb_train.cpu()), dim=2)
        draw_images(writer, train_batch[2].numpy(), concat_res_train.cpu().detach().numpy(), it, tag="train")
        
        concat_res_val = torch.cat((global_output_val.cpu(), local_output_val.cpu(), result_rgb_val.cpu()), dim=2)
        draw_images(writer, val_batch[2].numpy(), concat_res_val.cpu().detach().numpy(), it, tag="val")
    
    if ((it % save_models_every_it) == 0) and (it > 0):
        os.mkdir(os.path.join(checkpoints_folder, str(it)))
        torch.save(refinement_net, os.path.join(checkpoints_folder,  str(it), "refinement_net"))
        torch.save(refinement_net.state_dict(), os.path.join(checkpoints_folder,  str(it), "refinement_net_state_dict"))
        torch.save(local_transfer_net, os.path.join(checkpoints_folder,  str(it), "local_net"))
        torch.save(local_transfer_net.state_dict(), os.path.join(checkpoints_folder,  str(it), "local_net_state_dict"))
    
    if ((it % test_every_it) == 0) and (it > 0):
        refinement_result, local_result, global_result = inference_all_test_videos(test_videos_folders,
                              refinement_net, local_transfer_net, global_transferer,
                              save_test_output_videos_dir, it, use_only_local=pretrain_local,
                              save_result=True, resize_size=img_shape)
        gt_frames = load_all_test_videos(test_videos_folders, resize_size=img_shape)
        metric_value = calculate_psnr_result(gt_frames, refinement_result)
        draw_scalar_value(writer, "metrics", "test psnr", metric_value, it)
    

        

  0%|          | 0/10000 [00:00<?, ?it/s]
0it [00:00, ?it/s]

  0%|          | 0/90 [00:00<?, ?it/s]

  1%|          | 1/90 [00:01<02:11,  1.47s/it]

  2%|▏         | 2/90 [00:03<02:16,  1.55s/it]

  3%|▎         | 3/90 [00:04<02:11,  1.51s/it]

  4%|▍         | 4/90 [00:06<02:10,  1.52s/it]

  6%|▌         | 5/90 [00:07<02:07,  1.51s/it]

  7%|▋         | 6/90 [00:09<02:13,  1.59s/it]

  8%|▊         | 7/90 [00:11<02:11,  1.58s/it]

  9%|▉         | 8/90 [00:12<02:10,  1.59s/it]

 10%|█         | 9/90 [00:14<02:07,  1.58s/it]

 11%|█         | 10/90 [00:15<02:06,  1.58s/it]

 12%|█▏        | 11/90 [00:17<02:04,  1.57s/it]

 13%|█▎        | 12/90 [00:19<02:06,  1.62s/it]

 14%|█▍        | 13/90 [00:21<02:06,  1.64s/it]

 16%|█▌        | 14/90 [00:23<02:05,  1.65s/it]

 17%|█▋        | 15/90 [00:24<02:03,  1.65s/it]

 18%|█▊        | 16/90 [00:26<02:01,  1.64s/it]

 19%|█▉        | 17/90 [00:27<01:58,  1.63s/it]

 20%|██        | 18/90 [00:29<01:58,  1.64s/it]

 21%|██        | 19/90 [0

  2%|▏         | 1/59 [00:01<01:30,  1.57s/it]

  3%|▎         | 2/59 [00:03<01:35,  1.67s/it]

  5%|▌         | 3/59 [00:04<01:30,  1.61s/it]

  7%|▋         | 4/59 [00:06<01:27,  1.59s/it]

  8%|▊         | 5/59 [00:08<01:29,  1.66s/it]

 10%|█         | 6/59 [00:09<01:26,  1.64s/it]

 12%|█▏        | 7/59 [00:11<01:23,  1.61s/it]

 14%|█▎        | 8/59 [00:12<01:22,  1.62s/it]

 15%|█▌        | 9/59 [00:14<01:20,  1.61s/it]

 17%|█▋        | 10/59 [00:15<01:17,  1.59s/it]

 19%|█▊        | 11/59 [00:17<01:16,  1.59s/it]

 20%|██        | 12/59 [00:19<01:14,  1.59s/it]

 22%|██▏       | 13/59 [00:20<01:13,  1.59s/it]

 24%|██▎       | 14/59 [00:22<01:11,  1.59s/it]

 25%|██▌       | 15/59 [00:23<01:10,  1.59s/it]

 27%|██▋       | 16/59 [00:25<01:07,  1.58s/it]

 29%|██▉       | 17/59 [00:26<01:06,  1.58s/it]

 31%|███       | 18/59 [00:28<01:04,  1.58s/it]

 32%|███▏      | 19/59 [00:29<01:03,  1.58s/it]

 34%|███▍      | 20/59 [00:31<01:01,  1.59s/it]

 36%|███▌      | 21/59 [00:33

 24%|██▍       | 19/79 [00:31<01:39,  1.67s/it]

 25%|██▌       | 20/79 [00:33<01:38,  1.66s/it]

 27%|██▋       | 21/79 [00:35<01:37,  1.68s/it]

 28%|██▊       | 22/79 [00:36<01:35,  1.67s/it]

 29%|██▉       | 23/79 [00:38<01:33,  1.67s/it]

 30%|███       | 24/79 [00:39<01:31,  1.66s/it]

 32%|███▏      | 25/79 [00:41<01:29,  1.65s/it]

 33%|███▎      | 26/79 [00:42<01:27,  1.65s/it]

 34%|███▍      | 27/79 [00:44<01:25,  1.65s/it]

 35%|███▌      | 28/79 [00:46<01:24,  1.65s/it]

 37%|███▋      | 29/79 [00:48<01:22,  1.66s/it]

 38%|███▊      | 30/79 [00:50<01:21,  1.67s/it]

 39%|███▉      | 31/79 [00:51<01:19,  1.66s/it]

 41%|████      | 32/79 [00:53<01:17,  1.66s/it]

 42%|████▏     | 33/79 [00:54<01:16,  1.66s/it]

 43%|████▎     | 34/79 [00:56<01:14,  1.65s/it]

 44%|████▍     | 35/79 [00:57<01:12,  1.66s/it]

 46%|████▌     | 36/79 [00:59<01:11,  1.65s/it]

 47%|████▋     | 37/79 [01:00<01:09,  1.65s/it]

 48%|████▊     | 38/79 [01:02<01:07,  1.65s/it]

 49%|████▉     | 39/

 90%|█████████ | 81/90 [02:03<00:13,  1.53s/it]

 91%|█████████ | 82/90 [02:05<00:12,  1.53s/it]

 92%|█████████▏| 83/90 [02:07<00:10,  1.53s/it]

 93%|█████████▎| 84/90 [02:09<00:09,  1.54s/it]

 94%|█████████▍| 85/90 [02:10<00:07,  1.54s/it]

 96%|█████████▌| 86/90 [02:12<00:06,  1.54s/it]

 97%|█████████▋| 87/90 [02:13<00:04,  1.54s/it]

 98%|█████████▊| 88/90 [02:15<00:03,  1.54s/it]

 99%|█████████▉| 89/90 [02:16<00:01,  1.53s/it]

100%|██████████| 90/90 [02:17<00:00,  1.53s/it]


1it [02:18, 138.85s/it]

  0%|          | 0/54 [00:00<?, ?it/s]

  2%|▏         | 1/54 [00:01<01:25,  1.62s/it]

  4%|▎         | 2/54 [00:03<01:23,  1.60s/it]

  6%|▌         | 3/54 [00:04<01:14,  1.47s/it]

  7%|▋         | 4/54 [00:05<01:11,  1.42s/it]

  9%|▉         | 5/54 [00:07<01:11,  1.45s/it]

 11%|█         | 6/54 [00:08<01:08,  1.43s/it]

 13%|█▎        | 7/54 [00:10<01:07,  1.45s/it]

 15%|█▍        | 8/54 [00:11<01:08,  1.48s/it]

 17%|█▋        | 9/54 [00:13<01:06,  1.48s/it]

 19%|█▊     

 33%|███▎      | 23/69 [00:35<01:10,  1.53s/it]

 35%|███▍      | 24/69 [00:36<01:08,  1.53s/it]

 36%|███▌      | 25/69 [00:38<01:07,  1.54s/it]

 38%|███▊      | 26/69 [00:39<01:06,  1.54s/it]

 39%|███▉      | 27/69 [00:41<01:04,  1.53s/it]

 41%|████      | 28/69 [00:42<01:02,  1.53s/it]

 42%|████▏     | 29/69 [00:45<01:02,  1.56s/it]

 43%|████▎     | 30/69 [00:46<01:00,  1.55s/it]

 45%|████▍     | 31/69 [00:47<00:58,  1.54s/it]

 46%|████▋     | 32/69 [00:48<00:56,  1.53s/it]

 48%|████▊     | 33/69 [00:50<00:54,  1.53s/it]

 49%|████▉     | 34/69 [00:51<00:53,  1.52s/it]

 51%|█████     | 35/69 [00:53<00:52,  1.53s/it]

 52%|█████▏    | 36/69 [00:55<00:50,  1.53s/it]

 54%|█████▎    | 37/69 [00:56<00:49,  1.53s/it]

 55%|█████▌    | 38/69 [00:58<00:47,  1.53s/it]

 57%|█████▋    | 39/69 [00:59<00:45,  1.53s/it]

 58%|█████▊    | 40/69 [01:01<00:44,  1.53s/it]

 59%|█████▉    | 41/69 [01:03<00:43,  1.54s/it]

 61%|██████    | 42/69 [01:04<00:41,  1.54s/it]

 62%|██████▏   | 43/

...

 33%|███▎      | 18/54 [00:28<00:57,  1.59s/it]

 35%|███▌      | 19/54 [00:30<00:55,  1.58s/it]

 37%|███▋      | 20/54 [00:31<00:53,  1.58s/it]

 39%|███▉      | 21/54 [00:33<00:51,  1.57s/it]

 41%|████      | 22/54 [00:34<00:50,  1.58s/it]

 43%|████▎     | 23/54 [00:36<00:48,  1.58s/it]

 44%|████▍     | 24/54 [00:37<00:47,  1.58s/it]

 46%|████▋     | 25/54 [00:39<00:45,  1.58s/it]

 48%|████▊     | 26/54 [00:40<00:44,  1.57s/it]

 50%|█████     | 27/54 [00:42<00:42,  1.57s/it]

 52%|█████▏    | 28/54 [00:43<00:40,  1.57s/it]

 54%|█████▎    | 29/54 [00:45<00:39,  1.57s/it]

 56%|█████▌    | 30/54 [00:47<00:37,  1.57s/it]

 57%|█████▋    | 31/54 [00:48<00:36,  1.57s/it]

 59%|█████▉    | 32/54 [00:50<00:34,  1.57s/it]

 61%|██████    | 33/54 [00:51<00:32,  1.57s/it]

 63%|██████▎   | 34/54 [00:53<00:31,  1.57s/it]

 65%|██████▍   | 35/54 [00:54<00:29,  1.57s/it]

 67%|██████▋   | 36/54 [00:56<00:28,  1.57s/it]

 69%|██████▊   | 37/54 [00:58<00:26,  1.57s/it]

 70%|███████   | 38/

 74%|███████▍  | 51/69 [01:20<00:28,  1.58s/it]

 75%|███████▌  | 52/69 [01:22<00:26,  1.58s/it]

 77%|███████▋  | 53/69 [01:23<00:25,  1.58s/it]

 78%|███████▊  | 54/69 [01:25<00:23,  1.58s/it]

 80%|███████▉  | 55/69 [01:26<00:22,  1.58s/it]

 81%|████████  | 56/69 [01:28<00:20,  1.58s/it]

 83%|████████▎ | 57/69 [01:30<00:18,  1.58s/it]

 84%|████████▍ | 58/69 [01:32<00:17,  1.59s/it]

 86%|████████▌ | 59/69 [01:34<00:15,  1.59s/it]

 87%|████████▋ | 60/69 [01:35<00:14,  1.59s/it]

 88%|████████▊ | 61/69 [01:37<00:12,  1.60s/it]

 90%|████████▉ | 62/69 [01:38<00:11,  1.60s/it]

 91%|█████████▏| 63/69 [01:40<00:09,  1.60s/it]

 93%|█████████▎| 64/69 [01:42<00:07,  1.59s/it]

 94%|█████████▍| 65/69 [01:43<00:06,  1.59s/it]

 96%|█████████▌| 66/69 [01:45<00:04,  1.59s/it]

 97%|█████████▋| 67/69 [01:46<00:03,  1.59s/it]

 99%|█████████▊| 68/69 [01:48<00:01,  1.59s/it]

100%|██████████| 69/69 [01:49<00:00,  1.59s/it]


4it [07:12, 108.01s/it]

  0%|          | 0/79 [00:00<?, ?it/s]

  1

 48%|████▊     | 43/90 [01:07<01:13,  1.56s/it]

 49%|████▉     | 44/90 [01:08<01:11,  1.56s/it]

 50%|█████     | 45/90 [01:10<01:10,  1.56s/it]

 51%|█████     | 46/90 [01:11<01:08,  1.56s/it]

 52%|█████▏    | 47/90 [01:13<01:07,  1.56s/it]

 53%|█████▎    | 48/90 [01:14<01:05,  1.56s/it]

 54%|█████▍    | 49/90 [01:16<01:03,  1.56s/it]

 56%|█████▌    | 50/90 [01:18<01:02,  1.56s/it]

 57%|█████▋    | 51/90 [01:19<01:00,  1.56s/it]

 58%|█████▊    | 52/90 [01:21<00:59,  1.56s/it]

 59%|█████▉    | 53/90 [01:23<00:57,  1.57s/it]

 60%|██████    | 54/90 [01:24<00:56,  1.57s/it]

 61%|██████    | 55/90 [01:26<00:54,  1.57s/it]

 62%|██████▏   | 56/90 [01:27<00:53,  1.57s/it]

 63%|██████▎   | 57/90 [01:29<00:51,  1.57s/it]

 64%|██████▍   | 58/90 [01:30<00:50,  1.57s/it]

 66%|██████▌   | 59/90 [01:32<00:48,  1.57s/it]

 67%|██████▋   | 60/90 [01:34<00:47,  1.58s/it]

 68%|██████▊   | 61/90 [01:36<00:45,  1.58s/it]

 69%|██████▉   | 62/90 [01:38<00:44,  1.59s/it]

 70%|███████   | 63/

 76%|███████▋  | 45/59 [01:11<00:22,  1.59s/it]

 78%|███████▊  | 46/59 [01:13<00:20,  1.59s/it]

 80%|███████▉  | 47/59 [01:14<00:19,  1.59s/it]

 81%|████████▏ | 48/59 [01:16<00:17,  1.59s/it]

 83%|████████▎ | 49/59 [01:18<00:15,  1.59s/it]

 85%|████████▍ | 50/59 [01:19<00:14,  1.59s/it]

 86%|████████▋ | 51/59 [01:21<00:12,  1.59s/it]

 88%|████████▊ | 52/59 [01:22<00:11,  1.59s/it]

 90%|████████▉ | 53/59 [01:24<00:09,  1.59s/it]

 92%|█████████▏| 54/59 [01:26<00:07,  1.59s/it]

 93%|█████████▎| 55/59 [01:27<00:06,  1.59s/it]

 95%|█████████▍| 56/59 [01:28<00:04,  1.59s/it]

 97%|█████████▋| 57/59 [01:30<00:03,  1.59s/it]

 98%|█████████▊| 58/59 [01:33<00:01,  1.60s/it]

100%|██████████| 59/59 [01:34<00:00,  1.60s/it]


3it [05:28, 109.54s/it]

  0%|          | 0/69 [00:00<?, ?it/s]

  1%|▏         | 1/69 [00:01<02:07,  1.87s/it]

  3%|▎         | 2/69 [00:03<01:54,  1.70s/it]

  4%|▍         | 3/69 [00:05<01:52,  1.70s/it]

  6%|▌         | 4/69 [00:06<01:50,  1.70s/it]

  7%|▋ 

 80%|███████▉  | 63/79 [01:36<00:24,  1.54s/it]

 81%|████████  | 64/79 [01:38<00:23,  1.54s/it]

 82%|████████▏ | 65/79 [01:39<00:21,  1.54s/it]

 84%|████████▎ | 66/79 [01:41<00:20,  1.54s/it]

 85%|████████▍ | 67/79 [01:43<00:18,  1.54s/it]

 86%|████████▌ | 68/79 [01:44<00:16,  1.54s/it]

 87%|████████▋ | 69/79 [01:46<00:15,  1.54s/it]

 89%|████████▊ | 70/79 [01:47<00:13,  1.54s/it]

 90%|████████▉ | 71/79 [01:49<00:12,  1.54s/it]

 91%|█████████ | 72/79 [01:50<00:10,  1.54s/it]

 92%|█████████▏| 73/79 [01:52<00:09,  1.54s/it]

 94%|█████████▎| 74/79 [01:53<00:07,  1.54s/it]

 95%|█████████▍| 75/79 [01:55<00:06,  1.54s/it]

 96%|█████████▌| 76/79 [01:56<00:04,  1.53s/it]

 97%|█████████▋| 77/79 [01:58<00:03,  1.53s/it]

 99%|█████████▊| 78/79 [02:00<00:01,  1.54s/it]

100%|██████████| 79/79 [02:01<00:00,  1.54s/it]


5it [09:21, 112.26s/it]
  5%|▍         | 468/10000 [1:22:07<27:52:49, 10.53s/it]/opt/conda/lib/python3.6/site-packages/skimage/color/colorconv.py:997: UserWarning: Co

In [ ]:
videos_frames = load_all_test_videos(test_videos_folders, img_shape)